In [ ]:
# Data to Acquire:
# 1. overview of electric cars currently on the market from ev-database.org
# 2. Average car prices for consumers since 1960 (real and nominal)
# 3. US median wage since 1960 (real and nominal)
# 4. EV battery prices since 2013, can also be component prices (Li, Cobalt, Nickel, Aluminum)
# 5. EVs produced yearly since 2013
# 6. EVs sold yearly 2013

In [3]:
# Step 1: imports
from requests import get
from bs4 import BeautifulSoup
import os
import requests
import json
from typing import Dict, List, Optional, Union, cast
import time
import pandas as pd
from datetime import date

In [4]:
### Always Check for the Hidden API When Web Scraping - Youtube Video by John Watson Rooney

### Using REI's site as an example, under section camp electronics

# 1. navigate to site, inspect page elements
# 2. in the inspect page elemnets box, click the network tab in the toolbar at top, and check the box for 'Fetch/XHR' to retrieve the full web request
# 3. navigate to the bottom of the target page, click on 'more' or 'page 2', which will generate more search results
# 4. select the request that contains all of the web page info (page size/items). This result is the body of the scrpaing file we will build.
# 5. Copy the appropriate result with a right click "copy cURL"
# 6. Paste into Insomnia (API tool), and attempt to edit the item counts per page, to reduce aggregate page requests during scraping.
# 7. Once complete with parameter adjustment, click the drop down arrow to the right of the GET request, select 'generate code'
# 8. Select 'Python', copy to clipboard, paste in IDE
# 9. Note: common issues with generated dict:
#     a. quotes: 2x double quotes. fix by replacing the outer double quotes with single quotes
#     b. clean up the code by removing the empty payload varaible, then from the response variable equation

In [13]:
# do the basics, set up a get(url), check response
url = "https://ev-database.org/#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~450|range-slider-battery:prev~next=10~200|range-slider-towweight:prev~next=0~2500|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=9"
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
response = requests.get(url, headers)
soup = BeautifulSoup(response.text, 'html.parser')
soup.text[:50]
# functions check complete

'\n\n\n\n\n     \nCompare electric vehicles - EV Database'

In [271]:
# functions check
url = "https://ev-database.org/#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~450|range-slider-battery:prev~next=10~200|range-slider-towweight:prev~next=0~2500|range-slider-fastcharge:prev~next=0~1500|paging:currentPage=0|paging:number=all"
headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}
soup = BeautifulSoup(response.content, 'lxml')
soup.find_all('a', class_='title')[0]

<a class="title" href="/car/1708/MG-MG4-Electric-Long-Range"><span class="mg">MG</span> <span class="model">MG4 Electric Long Range</span></a>

In [276]:
testlink = 'https://ev-database.org/car/1708/MG-MG4-Electric-Long-Range'

r = requests.get(testlink, headers=headers)

soup = BeautifulSoup(r.content, 'html.parser')

title = soup.find(class_='sub-header').text

ev_range = soup.find('div', class_='in-line block', attrs='table')

print(title)
print(ev_range)

#print(title.replace("Battery Electric Vehicle",''))



MG MG4 Electric Long Range
Battery Electric Vehicle

None


In [255]:
def scrapeEVs(n):

    baseurl = 'https://ev-database.org'
    
    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

    evlinks = []

    for x in range(1,n):

        r = requests.get(f'https://ev-database.org/#sort:path~type~order=.rank~number~desc|range-slider-range:prev~next=0~1200|range-slider-acceleration:prev~next=2~23|range-slider-topspeed:prev~next=110~450|range-slider-battery:prev~next=10~200|range-slider-towweight:prev~next=0~2500|range-slider-fastcharge:prev~next=0~1500|paging:currentPage={x}|paging:number=36')

        soup = BeautifulSoup(r.content, 'lxml')
        #make sure this primary list is correctly accessed, b/c the secondary for loop will access the list child
        evlist = soup.find_all('div', class_='title-wrap')

        for ev in evlist:
            for link in ev.find_all('a', href=True):
                evlinks.append(baseurl + link['href'])
    




    #following line commented out for scrapes only 1 page in length
    #print(f"\rFetching page {page} of {n-1} at {url}, time per query is {time_taken}", end="")

    # soup = BeautifulSoup(response.text, 'html.parser')
    
    # make = [make.text for make in soup.find_all(class_="title")]
    # model = [model.text for model in soup.find_all(class_="title")]
    # accel = [accel.text for accel in soup.find_all(class_="acceleration")]
    # speed = [speed.text for speed in soup.find_all(class_="topspeed")]
    # ev_range = [ev_range.text for ev_range in soup.find_all(class_="erange_real")]
    # bev = [bev.text for bev in soup.find_all(class_="battery")]
    # charge = [charge.text for charge in soup.find_all(class_="fastcharge_speed_print")]
    # g_price = [g_price.text for g_price in soup.find_all(title="Price in Germany before incentives")]
    # d_price = [d_price.text for d_price in soup.find_all(title="Price in The Netherlands before incentives")]
    # uk_price = [uk_price.text for uk_price in soup.find_all(title="Price in the United Kingdom after incentives")]

    # evFacts = {'Make': make,
    #         'Model': model,
    #         '0_t0_100': accel,
    #         'Top_Speed': speed,
    #         'Range': ev_range,
    #         'Battery_Capacity': bev,
    #         'Fast_Charge': charge,
    #         'German_Price': g_price,
    #         'Dutch_Price': d_price,
    #         'UK_Price': uk_price}
    
    # results = results + evFacts
    # # use lists instead, then manually create the df columns
    # results_df = pd.DataFrame(results)
    # #results_df.to_csv('REIscrape.csv')
    # return results_df

In [ ]:
# Now I need to clean and transform this data
# step 1: seperate the 

In [246]:
import requests
from bs4 import BeautifulSoup

def scrapeWhiskey(n):

    baseurl = 'https://www.thewhiskyexchange.com/'

    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

    productlinks = []
    
    for x in range(1,n):

        r = requests.get(f'https://www.thewhiskyexchange.com/c/35/japanese-whisky?pg={x}')

        soup = BeautifulSoup(r.content, 'lxml')

        productlist = soup.find_all('li', class_="product-grid__item")

        for item in productlist:
            for link in item.find_all('a', href=True):
                productlinks.append(baseurl + link['href'])
    
    print(len(productlist), len(productlinks))

        


In [247]:
scrapeWhiskey(4)

4 52
